In [2]:
# load base model and tokenizer
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer

model_id = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# freeze model weights
for param in model.parameters():
    param.requires_grad = False

In [5]:
examples = [
    "The fresh powder on the mountain was incredible.",
    "Avoid the gaper gap at all costs.",
    "Après-ski is the best part of the day."
]

In [9]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(examples, return_tensors="pt", padding=True).to(model.device)
print("--- Tokenization Sanity Check ---")
print(f"Token IDs:\n{inputs['input_ids']}")
print(f"Decoded Text:\n{tokenizer.batch_decode(inputs['input_ids'], skip_special_tokens=True)}")

--- Tokenization Sanity Check ---
Token IDs:
tensor([[   785,   7722,  16723,    389,    279,  16301,    572,  15050,     13,
         151643, 151643, 151643],
        [ 52116,    279,    342,   3191,  12929,    518,    678,   7049,     13,
         151643, 151643, 151643],
        [ 20722,  12142,   1331,   6642,    374,    279,   1850,    949,    315,
            279,   1899,     13]], device='cuda:0')
Decoded Text:
['The fresh powder on the mountain was incredible.', 'Avoid the gaper gap at all costs.', 'Après-ski is the best part of the day.']


In [10]:
outputs = model(**inputs, output_hidden_states=True)
hidden_states = outputs.hidden_states
print("--- Hidden State Metadata ---")
print(f"Number of layers (including embedding): {len(hidden_states)}")
print(f"Shape of each layer tensor: {hidden_states[0].shape}") # [Batch, Seq_Len, Hidden_Size]

--- Hidden State Metadata ---
Number of layers (including embedding): 37
Shape of each layer tensor: torch.Size([3, 12, 2560])


In [12]:
last_layer = hidden_states[-1]
sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
final_token_vectors = last_layer[torch.arange(last_layer.size(0)), sequence_lengths]

In [13]:
mask = inputs['attention_mask'].unsqueeze(-1).expand(last_layer.size()).float()
sum_embeddings = torch.sum(last_layer * mask, dim=1)
sum_mask = torch.clamp(mask.sum(dim=1), min=1e-9)
mean_pooled_vectors = sum_embeddings / sum_mask

In [16]:
final_token_np = final_token_vectors.cpu().numpy()
mean_pooled_np = mean_pooled_vectors.cpu().numpy()
np.save("final_token_vectors.npy", final_token_np)
np.save("mean_pooled_vectors.npy", mean_pooled_np)

print("--- Extraction Complete ---")
print(f"Final Token Vectors Shape: {final_token_np.shape}")
print(f"Mean Pooled Vectors Shape: {mean_pooled_np.shape}")
print("Files saved: final_token_vectors.npy, mean_pooled_vectors.npy")

--- Extraction Complete ---
Final Token Vectors Shape: (3, 2560)
Mean Pooled Vectors Shape: (3, 2560)
Files saved: final_token_vectors.npy, mean_pooled_vectors.npy


In [17]:
model.eval()

Qwen3Model(
  (embed_tokens): Embedding(151936, 2560)
  (layers): ModuleList(
    (0-35): 36 x Qwen3DecoderLayer(
      (self_attn): Qwen3Attention(
        (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
        (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
        (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
        (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
        (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
      )
      (mlp): Qwen3MLP(
        (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
        (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
        (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
      (post_attention_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
    )
  )
  (norm): Qwen3RM

In [18]:
def extract_hidden_states(text: str) -> dict:
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    states = outputs.hidden_states
    hidden_states_dict = {
        f"layer_{i}": layer.squeeze(0).cpu().numpy()
        for i, layer in enumerate(states)
    }
    hidden_states_dict["input_tokens"] = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    return hidden_states_dict


In [19]:
data = extract_hidden_states("Skiing in the fresh powder.")
print(f"Layers extracted: {len(data) - 1}") # Minus 1 because of the 'input_tokens' key
print(f"Shape of last layer: {data['layer_12'].shape}") # [Seq_Len, Hidden_Dim]
print(f"Tokens: {data['input_tokens']}")

Layers extracted: 37
Shape of last layer: (8, 2560)
Tokens: ['S', 'ki', 'ing', 'Ġin', 'Ġthe', 'Ġfresh', 'Ġpowder', '.']
